In [23]:
import matplotlib.pyplot as plt
import numpy as np
from pandas.io.json import json_normalize
from football_modelling.plotting import createPitch
import json
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
import os


In [22]:
dotenv_location = find_dotenv()
load_dotenv(dotenv_location)


True

In [2]:
# Size of the pitch in yards (!!!)
pitchLengthX = 120
pitchWidthY = 80


In [3]:
# The team we are interested in
# team_required ="United States Women's"
team_required = "England Women's"
# team_required ="Sweden Women's"
# team_required ="Germany Women's"


In [27]:
# Get the list of matches
competition_id = 72
data_dir = Path(os.path.expanduser(os.getenv("DATA_ROOT_DIR")))
statsbomb_dir = data_dir.joinpath("open-data/data")

# Load the list of matches for this competition
matches_dir = statsbomb_dir.joinpath(f"matches/{competition_id}/30.json")
with open(matches_dir) as f:
    matches = json.load(f)


In [28]:
# Find the matches they played
match_id_required = []
for match in matches:
    home_team_name = match["home_team"]["home_team_name"]
    away_team_name = match["away_team"]["away_team_name"]
    if (home_team_name == team_required) or (away_team_name == team_required):
        match_id_required.append(match["match_id"])

match_id_required


[69301, 22936, 68337, 22962, 68362, 69199, 69258]

In [31]:
# Find the passes for each match
for ic, match_id in enumerate(match_id_required):

    # Load in all match events

    file_name = str(match_id) + ".json"
    with open(statsbomb_dir.joinpath("events").joinpath(file_name)) as data_file:
        data = json.load(data_file)
    df = json_normalize(data, sep="_").assign(match_id=file_name[:-5])
    team_actions = df["team_name"] == team_required
    df = df[team_actions]

    # A dataframe of passes
    passes_match = df.loc[df["type_name"] == "Pass"].set_index("id")
    # A dataframe of shots
    shots_match = df.loc[df["type_name"] == "Shot"].set_index("id")

    # Find shot times in seconds
    # This should be adjusted to account for overlapping halves of the match.
    shot_times = shots_match["minute"] * 60 + shots_match["second"]
    shot_window = 15
    shot_start = shot_times - shot_window
    pass_times = passes_match["minute"] * 60 + passes_match["second"]

    # Check with passes are whitin [shot_window] seconds of a shot
    # Idea from this code came from https://stackoverflow.com/questions/38201057/efficiently-check-if-value-is-present-in-any-of-given-ranges
    def in_range(pass_time, start, finish):
        return True in ((start < pass_time) & (pass_time < finish)).unique()

    pass_to_shot = pass_times.apply(lambda x: in_range(x, shot_start, shot_times))

    # Exclude corners
    iscorner = passes_match["pass_type_name"] == "Corner"

    danger_passes = passes_match[np.logical_and(pass_to_shot, np.logical_not(iscorner))]

    if ic == 0:
        passes = danger_passes
    else:
        passes = passes.append(danger_passes)

    print(
        "Match: "
        + str(match_id)
        + ". Number of danger passes is: "
        + str(len(danger_passes))
    )


/var/folders/nf/m6vxkd_s12jc7p45psfkvpxm0000gn/T/ipykernel_19425/1272950133.py:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(data, sep = "_").assign(match_id = file_name[:-5])


Match: 69301. Number of danger passes is: 25
Match: 22936. Number of danger passes is: 40
Match: 68337. Number of danger passes is: 42
Match: 22962. Number of danger passes is: 42
Match: 68362. Number of danger passes is: 22
Match: 69199. Number of danger passes is: 42
Match: 69258. Number of danger passes is: 18
